In [ ]:
import sys
import os

# Path to the project root (2 levels up from this notebook)
project_root = os.path.abspath("../../")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

# base langchain library imports
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

# tableau_langchain imports
from experimental.tools.calculate import initialize_generate_calc_string
from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa

# environment variables available to current process and sub processes
load_dotenv()
# variables for authenticating and interacting with a Tableau site
tableau_domain = os.environ['TABLEAU_DOMAIN']
tableau_site = os.environ['TABLEAU_SITE']
tableau_jwt_client_id = os.environ['TABLEAU_JWT_CLIENT_ID']
tableau_jwt_secret_id = os.environ['TABLEAU_JWT_SECRET_ID']
tableau_jwt_secret = os.environ['TABLEAU_JWT_SECRET']
tableau_api_version = os.environ['TABLEAU_API_VERSION']
tableau_user = os.environ['TABLEAU_USER']
# the target data source for this Tool
datasource_luid = os.environ['DATASOURCE_LUID']
# variables to control LLM models for the Agent and Tools
open_api_key = os.environ["OPENAI_API_KEY"]
agent_llm_model = os.environ['AGENT_MODEL']
tooling_llm_model = os.environ['TOOLING_MODEL']

In [ ]:
print('tableau_domain:', datasource_luid)

In [ ]:
# Initialize an LLM 
llm = ChatOpenAI(model='gpt-4o', temperature=0)

# Initalize simple_datasource_qa for querying Tableau Datasources through VDS
generate_calc = initialize_generate_calc_string(
    domain=tableau_domain,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model)

# load the List of Tools to be used by the Agent. In this case we will just load our data source Q&A tool.
tools = [ generate_calc ]

tableauHeadlessAgent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS, # Use OpenAI's function calling
    verbose = False)

# Run the agent
response = tableauHeadlessAgent.invoke("generate a calc to determine the first purchase date for each product")
response

In [4]:
from IPython.display import display, Markdown

# Initalize simple_datasource_qa for querying Tableau Datasources through VDS
generate_calc = initialize_generate_calc_string(
    domain=tableau_domain,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model)

analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_domain,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model)

tools = [generate_calc, analyze_datasource]

model = ChatOpenAI(model='o4-mini', temperature=1)

tableauAgent = create_react_agent(model, tools)

# Run the agent
messages = tableauAgent.invoke({"messages": [("human",'create a sales in euros calc which uses a conversion of 1.2 for USD to EUR and then use it to determine the sales per region.')]})
messages


ERROR:root:Unexpected error in get_headlessbi_data: Failed to query data source via Tableau VizQL Data Service. Status code: 400. Response: {"errorCode":"400802","message":"Error at 'query.fields.1': Additional property 'calculation' is not allowed. (code: 1000)\nFrom schema path: query.<#/components/schemas/Query>.fields.1.<items>.<#/components/schemas/Field>.<oneOf>.<additionalProperties>\nError at 'query.fields.1': Additional property 'function' is not allowed. (code: 1000)\nFrom schema path: query.<#/components/schemas/Query>.fields.1.<items>.<#/components/schemas/Field>.<oneOf>.<additionalProperties>","datetime":"2025-05-15T21:07:58.492833389Z"}


{'messages': [HumanMessage(content='create a sales in euros calc which uses a conversion of 1.2 for USD to EUR and then use it to determine the sales per region.', additional_kwargs={}, response_metadata={}, id='7c0c8ee1-415f-4a09-92b1-cc6861f58164'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f1E6ZkM1KwG9aLWZ5wpXKagx', 'function': {'arguments': '{"user_input":"Create a calculated field Sales (EUR) that converts Sales from USD to Euros by dividing [Sales] by 1.2."}', 'name': 'generate_calc_string'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 498, 'prompt_tokens': 703, 'total_tokens': 1201, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-BXa5gt7PfZRj1BvtyUnjsIrjhCOG7', 'f

In [11]:
display(Markdown(messages['messages'][4].content)) #display a nicely formatted answer for successful generations

Error: ToolException('\n                Tableau\'s VizQL Data Service return an error for the generated query:\n\n                {"fields":[{"fieldCaption":"Region","sortPriority":1},{"fieldCaption":"Sales","function":"SUM","fieldAlias":"Sales in Euros","calculation":"SUM([Sales]) / 1.2"}],"filters":[]}\n\n                The user_input used to write this query was:\n\n                Sales per region in Euros, where Sales (EUR) is calculated as [Sales] / 1.2.\n\n                This was the error:\n\n                An unexpected error occurred: Failed to query data source via Tableau VizQL Data Service. Status code: 400. Response: {"errorCode":"400802","message":"Error at \'query.fields.1\': Additional property \'calculation\' is not allowed. (code: 1000)\\nFrom schema path: query.<#/components/schemas/Query>.fields.1.<items>.<#/components/schemas/Field>.<oneOf>.<additionalProperties>\\nError at \'query.fields.1\': Additional property \'function\' is not allowed. (code: 1000)\\nFrom schema path: query.<#/components/schemas/Query>.fields.1.<items>.<#/components/schemas/Field>.<oneOf>.<additionalProperties>","datetime":"2025-05-15T21:07:58.492833389Z"}\n\n                Consider retrying this tool with the same inputs but include the previous query\n                causing the error and the error itself for the tool to correct itself on a retry.\n                If the error was an empty array, this usually indicates an incorrect filter value\n                was applied, thus returning no data\n                ')
 Please fix your mistakes.

In [13]:
from typing import Any, List, Dict

def extract_last_ai_message(result: Dict[str, Any]) -> str:
    """
    Given a result dict like you showed (with a 'messages' list),
    find the last AIMessage and return its .content.
    """
    messages: List[Any] = result.get("messages", [])
    # walk the list in reverse, pick the first AIMessage
    for msg in reversed(messages):
        if isinstance(msg, AIMessage):
            return msg.content
    raise ValueError("No AIMessage found in result")

In [15]:
Markdown(extract_last_ai_message(messages))

Here’s how you can do this in Tableau and the resulting figures from the Superstore datasource:

1. Create the calculated field  
   Name: Sales (EUR)  
   Formula:  
        [Sales] / 1.2  
   This divides every USD‐denominated sale by 1.2 to convert it into Euros.

2. Use that field to get total Sales (EUR) by Region. Below are the results:

Data Source: Superstore Datasource  
Maintainer: zhao.y@tableau.com

| Region  | Sales (EUR)    |
|---------|----------------|
| Central | 503,170.67 €   |
| East    | 691,828.17 €   |
| South   | 391,721.91 €   |
| West    | 739,813.61 €   |

These numbers are simply `SUM([Sales]) / 1.2` aggregated at the Region level.